In [2]:
import pandas as pd
import numpy as np

# CSV 파일 경로
file_path = 'Delta_PSM.csv'

# CSV 파일 불러오기
df = pd.read_csv(file_path)

# 계산할 컬럼 목록
columns_to_calculate = [
    'X_reg', 'Y_reg', 'pred_x', 'pred_y', 'residual_x', 'residual_y',
    'psm_fit_x', 'psm_fit_y', 'residual_x_depsm', 'residual_y_depsm',
    'cpe19p_pred_x', 'cpe19p_pred_y', 'cpe19p_resi_x', 'cpe19p_resi_y',
    'ideal_psm_x', 'ideal_psm_y', 'delta_psm_x', 'delta_psm_y'
]

# 반지름 계산
df['radius'] = np.sqrt(df['wf_x']**2 + df['wf_y']**2)

# 영역 할당 (Outside 제거)
conditions = [
    (df['radius'] <= 50000),
    (df['radius'] > 50000) & (df['radius'] <= 100000),
    (df['radius'] > 100000)
]
choices = ['Center', 'Middle', 'Edge']
df['region'] = np.select(conditions, choices)

# 'UNIQUE_ID'와 'region' 기준으로 그룹화하여 mean과 std 계산
grouped = df.groupby(['UNIQUE_ID', 'region']).agg({col: ['mean', 'std'] for col in columns_to_calculate})

# 컬럼명 재정의 (멀티인덱스를 단일 인덱스로 변환)
grouped.columns = [f'{col}_{stat}' for col, stat in grouped.columns]

# mean에 절대값 적용
for col in columns_to_calculate:
    grouped[f'{col}_mean'] = grouped[f'{col}_mean'].abs()

# mean + 3 * sigma 계산
for col in columns_to_calculate:
    grouped[f'{col}_m3s'] = grouped[f'{col}_mean'] + 3 * grouped[f'{col}_std']

# 그룹별 첫 번째 값 추가
first_values = df.groupby(['UNIQUE_ID', 'region']).first().reset_index()
grouped = grouped.reset_index()
grouped = grouped.merge(first_values[['UNIQUE_ID', 'region', 'STEPSEQ', 'LOT_ID', 'Wafer', 'P_EQPID', 'Photo_PPID',
                                      'P_TIME', 'M_TIME', 'ChuckID', 'ReticleID', 'Base_EQP1', 'GROUP', 'MMO_MRC_EQP']],
                        on=['UNIQUE_ID', 'region'], how='left')

# 'All' 영역 추가 ('UNIQUE_ID' 기준으로 전체 데이터 그룹화)
grouped_all = df.groupby('UNIQUE_ID').agg({col: ['mean', 'std'] for col in columns_to_calculate})

# 컬럼명 재정의
grouped_all.columns = [f'{col}_{stat}' for col, stat in grouped_all.columns]

# mean에 절대값 적용
for col in columns_to_calculate:
    grouped_all[f'{col}_mean'] = grouped_all[f'{col}_mean'].abs()

# mean + 3 * sigma 계산
for col in columns_to_calculate:
    grouped_all[f'{col}_m3s'] = grouped_all[f'{col}_mean'] + 3 * grouped_all[f'{col}_std']

# 그룹별 첫 번째 값 추가
first_values_all = df.groupby('UNIQUE_ID').first().reset_index()
grouped_all = grouped_all.reset_index()
grouped_all = grouped_all.merge(first_values_all[['UNIQUE_ID', 'STEPSEQ', 'LOT_ID', 'Wafer', 'P_EQPID', 'Photo_PPID',
                                                  'P_TIME', 'M_TIME', 'ChuckID', 'ReticleID', 'Base_EQP1', 'GROUP', 'MMO_MRC_EQP']],
                                on='UNIQUE_ID', how='left')

# 'region' 컬럼 추가 및 'All' 할당
grouped_all['region'] = 'All'

# 그룹 데이터 병합
grouped_combined = pd.concat([grouped, grouped_all], ignore_index=True)

# 원하는 컬럼 순서 정의
desired_column_order = [
    'UNIQUE_ID', 'region', 'STEPSEQ', 'LOT_ID', 'Wafer', 'P_EQPID', 'Photo_PPID', 'P_TIME',
    'M_TIME', 'ChuckID', 'ReticleID', 'Base_EQP1', 'MMO_MRC_EQP', 'GROUP'
]

# 계산된 컬럼 추가
for col in columns_to_calculate:
    desired_column_order.extend([
        f'{col}_mean',
        f'{col}_std',
        f'{col}_m3s'
    ])

# DataFrame의 컬럼을 재배열
grouped_combined = grouped_combined[desired_column_order]

# 결과를 CSV 파일로 저장
output_file_path = 'output_m3s_by_region.csv'
grouped_combined.to_csv(output_file_path, index=False)

print("영역별로 m3s 계산 완료 및 저장됨:", output_file_path)


영역별로 m3s 계산 완료 및 저장됨: output_m3s_by_region.csv
